In [124]:
library(igraph)
library(ggplot2)

In [125]:
# costruct network from data
graphWithLabel = read_graph("./facebook_combined.txt", format="edgelist", directed=FALSE)
graphWithLabel = set_vertex_attr(graphWithLabel, name = "label", index = seq(1, length(V(graphWithLabel))), value = seq(1, length(V(graphWithLabel))))

### Q16

In [126]:
# personalized network for node 415
eg = make_ego_graph(graph = graphWithLabel, order = 1, nodes = V(graphWithLabel)[415])[[1]]
Nr = which(degree(eg)==24)
print(sprintf("Length of Nr = %d", length(Nr)))

[1] "Length of Nr = 11"


### Q17

In [127]:
commonNeighbor = function(graph, ni, nj) {
    return(length(intersect(neighbors(graph = graph, v = ni), neighbors(graph = graph, v = nj))))
}

In [128]:
jaccard = function(graph, ni, nj) {
    return(commonNeighbor(graph, ni, nj)/length(union(neighbors(graph = graph, v = ni), neighbors(graph = graph, v = nj))))
}

In [129]:
adamic = function(graph, ni, nj) {
    ret = 0
    for (node in intersect(neighbors(graph = graph, v = ni), neighbors(graph = graph, v = nj))) {
        ret = ret + 1/log10(length(neighbors(graph = graph, v = node)))
    }
    return(ret)
}

In [159]:
recommend = function(graph, ni, alg) {
    dice = c(TRUE, FALSE, FALSE, FALSE)
    
    # unfriend
    removeEdges = c()
    removedFriends = c()
    for (edge in incident(graph, ni)) {
        if (sample(dice, 1)) {
            removeEdges = c(removeEdges, edge)
            if (ni == tail_of(graph, edge)) {
                removedFriends = c(removedFriends, head_of(graph, edge))
            }
            else {
                removedFriends = c(removedFriends, tail_of(graph, edge))
            }
        }
    }
    graph = delete_edges(graph, removeEdges)
    
    # friend
    notFriends = c()
    friends = neighbors(graph, ni)
    for (v in V(graph)) {
        if (!(v %in% friends)) {
            notFriends = c(notFriends, v)
        }
    }
    score = c()
    for (nj in notFriends) {
        score = c(score, alg(graph, ni, nj))
    }
    score = sort(score, decreasing = TRUE, index.return = TRUE)
    
    candidates = notFriends[score$ix]
    recommendation = candidates[1:length(removedFriends)]
    
    # calculate average
    acc = length(intersect(removedFriends, recommendation))/length(removedFriends)
    return(acc)
}

In [160]:
runExp = function(alg) {
    aveGlobal = 0
    for (node in Nr) {
        aveLocal = 0
        for (i in 1:10) {
            aveLocal = aveLocal + recommend(eg, node, alg)
        }
        aveLocal = aveLocal / 10
        aveGlobal = aveGlobal + aveLocal
    }
    aveGlobal = aveGlobal / length(Nr)
    return(aveGlobal)
}
print(sprintf("Accuracy for Common Neighbor: %.2f", runExp(commonNeighbor)))
print(sprintf("Accuracy for Jaccard: %.2f", runExp(jaccard)))
print(sprintf("Accuracy for Adamic Adar: %.2f", runExp(adamic)))

[1] "Accuracy for Common Neighbor: 0.72"
[1] "Accuracy for Jaccard: 0.68"
[1] "Accuracy for Adamic Adar: 0.70"
